In [1]:
import numpy as np
import itertools
import random

# Написать функцию формирования порождающей и проверочной матриц кода Хэммин на основе параметра r, а также таблицы синдромов для всех однократных ошибок.


In [2]:
# Функция формирования проверочной матрицы
def generate_hamming_h_matrix(r: int) -> np.ndarray:
    n = 2 ** r - 1
    res = []
    cur_r = r - 1
    for i in range(n, 0, -1):
        if i != 2 ** cur_r:
            res.append(list(map(int, f"{i:0{r}b}")))
        else:
            cur_r -= 1

    identity_matrix = np.eye(r, dtype=int)

    H = np.vstack((res, identity_matrix))

    return H


# Функция построения порождающей матрицы
def H_to_G(H: np.ndarray, r) -> np.ndarray:
    k = 2 ** r - r - 1
    res = np.eye(k, dtype=int)
    G = np.hstack((res, H[:k]))
    return G


# Функция построения таблицы синдромов
def generate_syndrome_table(matrix: np.ndarray, error_weight: int) -> dict:
    n = matrix.shape[0]
    syndrome_table = {}
    for error in range(1, error_weight + 1):
        for error_indices in itertools.combinations(range(n), error):
            error_vector = np.zeros(n, dtype=int)
            for index in error_indices:
                error_vector[index] = 1
            syndrome = error_vector @ matrix % 2
            syndrome_table[tuple(map(int, syndrome))] = tuple(error_indices)

    return syndrome_table

## Проверочная матрица при r = 2

In [3]:
r = 2
H = generate_hamming_h_matrix(r)
H

array([[1, 1],
       [1, 0],
       [0, 1]])

## Порождающая матрица на основе проверочной матрицы

In [4]:
G = H_to_G(H, r)
G

array([[1, 1, 1]])

## Таблица синдромов для всех единичных ошибок

In [5]:
syndrome_table = generate_syndrome_table(H, 1)
syndrome_table

{(1, 1): (0,), (1, 0): (1,), (0, 1): (2,)}

# Провести исследование кода Хэмминга для одно-, двух- и трёхкратных ошибок для r = 2, 3, 4.

In [40]:
# Функция для допущения и проверки ошибки

def hamming_correction_test(G: np.ndarray, H: np.ndarray, syndrome_table: dict, error_degree: int, u: np.ndarray):
    # Шаг 1: Формируем кодовое слово u и кодируем его с помощью G
    print(f"Исходное слово: {u}")
    encoded_word = (u @ G) % 2
    print(f"Закодированное слово: {encoded_word}")

    # Шаг 2: Вносим ошибки в кодовое слово
    error_pattern = np.zeros_like(encoded_word, dtype=int)
    error_positions = random.sample(range(len(encoded_word)), error_degree)
    error_pattern[error_positions] = 1
    print(f"Ошибка: {error_pattern}")

    # Слово с ошибками
    received_word = (encoded_word + error_pattern) % 2
    print(f"Принятое слово с ошибкой: {received_word}")

    # Шаг 3: Рассчитываем синдром
    syndrome = (received_word @ H) % 2
    print(f"Синдром: {syndrome}")

    # Шаг 4: Если синдром ненулевой, проводим коррекцию
    if np.any(syndrome):
        print("Ошибка обнаружена.")
        error_position = syndrome_table.get(tuple(syndrome), None)

        # Если синдром найден в таблице, корректируем ошибку
        if error_position:
            received_word[error_position] = (received_word[error_position] + 1) % 2
            print(f"Сообщение после исправления: {received_word}")

            # Проверка корректности исправления
            if np.array_equal(encoded_word, received_word):
                print("Ошибка успешно исправлена!")
            else:
                print("Ошибка не была полностью исправлена.")
        else:
            print("Синдром отсутствует в таблице, ошибка не исправлена.")
    else:
        print("Ошибок не обнаружено.")

## Проведем исследование кода Хэмминга для ошибки кратности 1

In [7]:
hamming_correction_test(G, H, syndrome_table, 1, np.array([1]))

Исходное слово: [1]
Закодированное слово: [1 1 1]
Ошибка: [0 0 1]
Принятое слово с ошибкой: [1 1 0]
Синдром: [0 1]
Ошибка обнаружена.
Сообщение после исправления: [1 1 1]
Ошибка успешно исправлена!


Код успешно нашел ошибку и исправил ее

## Проведем исследование кода Хэмминга для ошибки кратности 2

In [8]:
hamming_correction_test(G, H, syndrome_table, 2, np.array([1]))

Исходное слово: [1]
Закодированное слово: [1 1 1]
Ошибка: [1 0 1]
Принятое слово с ошибкой: [0 1 0]
Синдром: [1 0]
Ошибка обнаружена.
Сообщение после исправления: [0 0 0]
Ошибка не была полностью исправлена.


Код смог обнаружить ошибку, но не смогли ее исправить

## Проведем исследование кода Хэмминга для ошибки кратности 3

In [9]:
hamming_correction_test(G, H, syndrome_table, 3, np.array([1]))

Исходное слово: [1]
Закодированное слово: [1 1 1]
Ошибка: [1 1 1]
Принятое слово с ошибкой: [0 0 0]
Синдром: [0 0]
Ошибок не обнаружено.


Код не смог даже обнаружить ошибку

# Теперь проделаем все для r = 3, но для начала создадим:
## 1. Проверочная матрица при r = 3
## 2. Порождающая матрица на основе проверочной матрицы
## 3. Таблица синдромов

In [10]:
# 1
r = 3
H = generate_hamming_h_matrix(r)
H

array([[1, 1, 1],
       [1, 1, 0],
       [1, 0, 1],
       [0, 1, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [11]:
#2
G = H_to_G(H, r)
G

array([[1, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 1, 1]])

In [12]:
#3
syndrome_table = generate_syndrome_table(H, 1)
syndrome_table

{(1, 1, 1): (0,),
 (1, 1, 0): (1,),
 (1, 0, 1): (2,),
 (0, 1, 1): (3,),
 (1, 0, 0): (4,),
 (0, 1, 0): (5,),
 (0, 0, 1): (6,)}

## Проведем исследование кода Хэмминга для ошибки кратности 1

In [13]:
hamming_correction_test(G, H, syndrome_table, 1, np.array([1, 0, 0, 1]))

Исходное слово: [1 0 0 1]
Закодированное слово: [1 0 0 1 1 0 0]
Ошибка: [0 0 0 0 1 0 0]
Принятое слово с ошибкой: [1 0 0 1 0 0 0]
Синдром: [1 0 0]
Ошибка обнаружена.
Сообщение после исправления: [1 0 0 1 1 0 0]
Ошибка успешно исправлена!


Код успешно нашел ошибку и исправил ее

## Проведем исследование кода Хэмминга для ошибки кратности 2

In [14]:
hamming_correction_test(G, H, syndrome_table, 2, np.array([1, 0, 0, 1]))

Исходное слово: [1 0 0 1]
Закодированное слово: [1 0 0 1 1 0 0]
Ошибка: [0 1 0 0 0 0 1]
Принятое слово с ошибкой: [1 1 0 1 1 0 1]
Синдром: [1 1 1]
Ошибка обнаружена.
Сообщение после исправления: [0 1 0 1 1 0 1]
Ошибка не была полностью исправлена.


Код смог обнаружить ошибку, но не смог ее исправить

## Проведем исследование кода Хэмминга для ошибки кратности 3

In [15]:
hamming_correction_test(G, H, syndrome_table, 3, np.array([1, 0, 0, 1]))

Исходное слово: [1 0 0 1]
Закодированное слово: [1 0 0 1 1 0 0]
Ошибка: [0 1 0 0 1 1 0]
Принятое слово с ошибкой: [1 1 0 1 0 1 0]
Синдром: [0 0 0]
Ошибок не обнаружено.


Код не обнаружил ошибки

# Теперь проделаем все для r = 4, но для начала создадим:
## 1. Проверочная матрица при r = 4
## 2. Порождающая матрица на основе проверочной матрицы
## 3. Таблица синдромов

In [16]:
# 1
r = 4
H = generate_hamming_h_matrix(r)
H

array([[1, 1, 1, 1],
       [1, 1, 1, 0],
       [1, 1, 0, 1],
       [1, 1, 0, 0],
       [1, 0, 1, 1],
       [1, 0, 1, 0],
       [1, 0, 0, 1],
       [0, 1, 1, 1],
       [0, 1, 1, 0],
       [0, 1, 0, 1],
       [0, 0, 1, 1],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [17]:
#2
G = H_to_G(H, r)
G

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1]])

In [18]:
#3
syndrome_table = generate_syndrome_table(H, 1)
syndrome_table

{(1, 1, 1, 1): (0,),
 (1, 1, 1, 0): (1,),
 (1, 1, 0, 1): (2,),
 (1, 1, 0, 0): (3,),
 (1, 0, 1, 1): (4,),
 (1, 0, 1, 0): (5,),
 (1, 0, 0, 1): (6,),
 (0, 1, 1, 1): (7,),
 (0, 1, 1, 0): (8,),
 (0, 1, 0, 1): (9,),
 (0, 0, 1, 1): (10,),
 (1, 0, 0, 0): (11,),
 (0, 1, 0, 0): (12,),
 (0, 0, 1, 0): (13,),
 (0, 0, 0, 1): (14,)}

## Проведем исследование кода Хэмминга для ошибки кратности 1

In [19]:
hamming_correction_test(G, H, syndrome_table, 1, np.array([0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1]))

Исходное слово: [0 0 1 0 1 1 0 0 1 1 1]
Закодированное слово: [0 0 1 0 1 1 0 0 1 1 1 1 1 0 0]
Ошибка: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Принятое слово с ошибкой: [0 0 1 0 1 1 0 0 1 1 0 1 1 0 0]
Синдром: [0 0 1 1]
Ошибка обнаружена.
Сообщение после исправления: [0 0 1 0 1 1 0 0 1 1 1 1 1 0 0]
Ошибка успешно исправлена!


Код успешно нашел ошибку и исправил ее

## Проведем исследование кода Хэмминга для ошибки кратности 2

In [20]:
hamming_correction_test(G, H, syndrome_table, 2, np.array([0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1]))

Исходное слово: [0 0 1 0 1 1 0 0 1 1 1]
Закодированное слово: [0 0 1 0 1 1 0 0 1 1 1 1 1 0 0]
Ошибка: [0 0 0 0 0 0 0 0 0 0 1 0 1 0 0]
Принятое слово с ошибкой: [0 0 1 0 1 1 0 0 1 1 0 1 0 0 0]
Синдром: [0 1 1 1]
Ошибка обнаружена.
Сообщение после исправления: [0 0 1 0 1 1 0 1 1 1 0 1 0 0 0]
Ошибка не была полностью исправлена.


Код смог обнаружить ошибку, но не смог ее исправить

## Проведем исследование кода Хэмминга для ошибки кратности 3

In [21]:
hamming_correction_test(G, H, syndrome_table, 3, np.array([0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1]))

Исходное слово: [0 0 1 0 1 1 0 0 1 1 1]
Закодированное слово: [0 0 1 0 1 1 0 0 1 1 1 1 1 0 0]
Ошибка: [0 0 0 0 0 0 0 0 1 1 0 0 0 1 0]
Принятое слово с ошибкой: [0 0 1 0 1 1 0 0 0 0 1 1 1 1 0]
Синдром: [0 0 0 1]
Ошибка обнаружена.
Сообщение после исправления: [0 0 1 0 1 1 0 0 0 0 1 1 1 1 1]
Ошибка не была полностью исправлена.


Код смог обнаружить ошибку, но не смог ее исправить

# Написать функцию формирования порождающей и проверочной матриц расширенного кода Хэмминга на основе параметра r, а также таблицы синдромов для всех однократных ошибок

In [22]:
# Возьмем r = 3
r = 2
H = generate_hamming_h_matrix(r)
H_exp = np.vstack((H, np.array([0] * H.shape[1])))
H_exp = np.hstack((H_exp, np.array([[1] * H_exp.shape[0]]).T))
H_exp

array([[1, 1, 1],
       [1, 0, 1],
       [0, 1, 1],
       [0, 0, 1]])

In [23]:
# Функция для параждающей матрицы
def expand_G_matrix(G: np.ndarray) -> np.ndarray:
    col = np.zeros((G.shape[0], 1), dtype=int)
    for i in range(G.shape[0]):
        if sum(G[i]) % 2 == 1:
            col[i] = 1
    return np.hstack((G, col))


# Построим
G = H_to_G(H, r)
G_exp = expand_G_matrix(G)
G_exp

array([[1, 1, 1, 1]])

In [24]:
# Таблица синдромов
syndrome_table = generate_syndrome_table(H_exp, 1)
syndrome_table

{(1, 1, 1): (0,), (1, 0, 1): (1,), (0, 1, 1): (2,), (0, 0, 1): (3,)}

# Провести исследование расширенного кода Хэмминга для одно-, двух-, трёх- и четырёхкратных ошибок для r = 2, 3, 4

In [25]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 1, np.array([1]))

Исходное слово: [1]
Закодированное слово: [1 1 1 1]
Ошибка: [0 0 1 0]
Принятое слово с ошибкой: [1 1 0 1]
Синдром: [0 1 1]
Ошибка обнаружена.
Сообщение после исправления: [1 1 1 1]
Ошибка успешно исправлена!


In [26]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 2, np.array([1]))

Исходное слово: [1]
Закодированное слово: [1 1 1 1]
Ошибка: [0 1 1 0]
Принятое слово с ошибкой: [1 0 0 1]
Синдром: [1 1 0]
Ошибка обнаружена.
Синдром отсутствует в таблице, ошибка не исправлена.


In [27]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 3, np.array([1]))

Исходное слово: [1]
Закодированное слово: [1 1 1 1]
Ошибка: [0 1 1 1]
Принятое слово с ошибкой: [1 0 0 0]
Синдром: [1 1 1]
Ошибка обнаружена.
Сообщение после исправления: [0 0 0 0]
Ошибка не была полностью исправлена.


In [28]:
r = 3
H = generate_hamming_h_matrix(r)
H_exp = np.vstack((H, np.array([0] * H.shape[1])))
H_exp = np.hstack((H_exp, np.array([[1] * H_exp.shape[0]]).T))
H_exp

array([[1, 1, 1, 1],
       [1, 1, 0, 1],
       [1, 0, 1, 1],
       [0, 1, 1, 1],
       [1, 0, 0, 1],
       [0, 1, 0, 1],
       [0, 0, 1, 1],
       [0, 0, 0, 1]])

In [29]:
G = H_to_G(H, r)
G_exp = expand_G_matrix(G)
G_exp

array([[1, 0, 0, 0, 1, 1, 1, 0],
       [0, 1, 0, 0, 1, 1, 0, 1],
       [0, 0, 1, 0, 1, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 1, 1]])

In [30]:
syndrome_table = generate_syndrome_table(H_exp, 1)
syndrome_table

{(1, 1, 1, 1): (0,),
 (1, 1, 0, 1): (1,),
 (1, 0, 1, 1): (2,),
 (0, 1, 1, 1): (3,),
 (1, 0, 0, 1): (4,),
 (0, 1, 0, 1): (5,),
 (0, 0, 1, 1): (6,),
 (0, 0, 0, 1): (7,)}

In [31]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 1, np.array([1, 0, 1, 0]))

Исходное слово: [1 0 1 0]
Закодированное слово: [1 0 1 0 0 1 0 1]
Ошибка: [0 0 0 0 1 0 0 0]
Принятое слово с ошибкой: [1 0 1 0 1 1 0 1]
Синдром: [1 0 0 1]
Ошибка обнаружена.
Сообщение после исправления: [1 0 1 0 0 1 0 1]
Ошибка успешно исправлена!


In [32]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 2, np.array([1, 0, 1, 0]))

Исходное слово: [1 0 1 0]
Закодированное слово: [1 0 1 0 0 1 0 1]
Ошибка: [0 1 0 0 1 0 0 0]
Принятое слово с ошибкой: [1 1 1 0 1 1 0 1]
Синдром: [0 1 0 0]
Ошибка обнаружена.
Синдром отсутствует в таблице, ошибка не исправлена.


In [33]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 3, np.array([1, 0, 1, 0]))

Исходное слово: [1 0 1 0]
Закодированное слово: [1 0 1 0 0 1 0 1]
Ошибка: [1 1 0 0 0 1 0 0]
Принятое слово с ошибкой: [0 1 1 0 0 0 0 1]
Синдром: [0 1 1 1]
Ошибка обнаружена.
Сообщение после исправления: [0 1 1 1 0 0 0 1]
Ошибка не была полностью исправлена.


In [34]:
r = 4
H = generate_hamming_h_matrix(r)
H_exp = np.vstack((H, np.array([0] * H.shape[1])))
H_exp = np.hstack((H_exp, np.array([[1] * H_exp.shape[0]]).T))
H_exp

array([[1, 1, 1, 1, 1],
       [1, 1, 1, 0, 1],
       [1, 1, 0, 1, 1],
       [1, 1, 0, 0, 1],
       [1, 0, 1, 1, 1],
       [1, 0, 1, 0, 1],
       [1, 0, 0, 1, 1],
       [0, 1, 1, 1, 1],
       [0, 1, 1, 0, 1],
       [0, 1, 0, 1, 1],
       [0, 0, 1, 1, 1],
       [1, 0, 0, 0, 1],
       [0, 1, 0, 0, 1],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1]])

In [35]:
G = H_to_G(H, r)
G_exp = expand_G_matrix(G)
G_exp

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1]])

In [36]:
syndrome_table = generate_syndrome_table(H_exp, 1)
syndrome_table

{(1, 1, 1, 1, 1): (0,),
 (1, 1, 1, 0, 1): (1,),
 (1, 1, 0, 1, 1): (2,),
 (1, 1, 0, 0, 1): (3,),
 (1, 0, 1, 1, 1): (4,),
 (1, 0, 1, 0, 1): (5,),
 (1, 0, 0, 1, 1): (6,),
 (0, 1, 1, 1, 1): (7,),
 (0, 1, 1, 0, 1): (8,),
 (0, 1, 0, 1, 1): (9,),
 (0, 0, 1, 1, 1): (10,),
 (1, 0, 0, 0, 1): (11,),
 (0, 1, 0, 0, 1): (12,),
 (0, 0, 1, 0, 1): (13,),
 (0, 0, 0, 1, 1): (14,),
 (0, 0, 0, 0, 1): (15,)}

In [37]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 1, np.array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1,0]))

Исходное слово: [1 0 0 1 1 1 1 0 1 1 0]
Закодированное слово: [1 0 0 1 1 1 1 0 1 1 0 1 0 0 0 0]
Ошибка: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Принятое слово с ошибкой: [1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 0]
Синдром: [1 0 0 0 1]
Ошибка обнаружена.
Сообщение после исправления: [1 0 0 1 1 1 1 0 1 1 0 1 0 0 0 0]
Ошибка успешно исправлена!


In [38]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 2, np.array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1,0]))

Исходное слово: [1 0 0 1 1 1 1 0 1 1 0]
Закодированное слово: [1 0 0 1 1 1 1 0 1 1 0 1 0 0 0 0]
Ошибка: [0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Принятое слово с ошибкой: [1 1 0 1 1 1 0 0 1 1 0 1 0 0 0 0]
Синдром: [0 1 1 1 0]
Ошибка обнаружена.
Синдром отсутствует в таблице, ошибка не исправлена.


In [39]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 3, np.array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1,0]))

Исходное слово: [1 0 0 1 1 1 1 0 1 1 0]
Закодированное слово: [1 0 0 1 1 1 1 0 1 1 0 1 0 0 0 0]
Ошибка: [1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]
Принятое слово с ошибкой: [0 0 0 1 1 1 1 0 1 0 0 1 0 0 0 1]
Синдром: [1 0 1 0 1]
Ошибка обнаружена.
Сообщение после исправления: [0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 1]
Ошибка не была полностью исправлена.


## Получили результаты аналогичные для всех r. Расширенный код Хэмминга позволяет исправлять однократные ошибки и обнаруживать двухкратные одновременно.